In [ ]:
import pandas as pd
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import os
import math

sr = 200

print(f"CWD: {os.getcwd()}")

In [ ]:
def get_qom(x: float, y: float, z: float) -> float:
    return np.std(math.sqrt((x**2 + y**2 + z**2) / 3))

In [ ]:
accel_root_dir = "./accel_data/"
groups = ["school", "staff", "public"] 

qom_dir = "./qom_data/"

columns = ["ts", "x", "y", "z"] # Dataframe columns

# Map sensor IDs (in filenames) to locations numbers - comments are headphone numbers
id_code_mappings = {
    "17515" : "01",  # 11
    "17510" : "02",  # 12
    "14452" : "03",  # 13 - labelled BAD
    "14266" : "04",  # 14
    "14357" : "05",  # 15
    "14083" : "06",  # 16
    "13441" : "07",  # 17
    "14155" : "08",  # 18
    "15898" : "09",  # 19
    "14819" : "10",  # 20
    "14564" : "11",  # 1
    "14341" : "12"   # 2
}

folder_round_mappings = {
    "school" : "A",
    "staff"  : "B",
    "public" : "CDEFGHI"
}

for group in groups:
    dir_path = os.path.join(accel_root_dir, group)

    csv_filenames = [file for file in os.listdir(dir_path) if file.endswith(".csv")]

    for csv_filename in csv_filenames:
        csv_path = os.path.join(dir_path, csv_filename)

        print(f"Loading: {csv_path}")
        accel_df = pd.read_csv(csv_path, names=columns)
        print("Loaded CSV.")
        
        qom_df = accel_df.copy()
        
        # Get QoM
        print(f"  Calculating QoM for {len(accel_df.index)} rows...")
        for i in range(len(accel_df.index)):
            if i % 10000 == 0 and i > 0:
                print(f"    Up to row {i}...")
            accel_row = accel_df.iloc[i]
            qom_df.loc[i, "qom"] = get_qom(accel_row.loc["x"], accel_row.loc["y"], accel_row.loc["z"])
            # qom_df.loc[i, "ts"] = accel_row.loc["ts"]
        qom_df.drop(["x", "y", "z"], axis=1, inplace=True)
        print("  Added QoM to dataframe.")

        # Get file identifiers
        sensor_id = csv_filename[:5]
        headphone_location = id_code_mappings[sensor_id]
        included_rounds = folder_round_mappings[group]

        # Create save filename
        save_filename = f"{headphone_location}_{included_rounds}_{sensor_id}.csv"
        save_filepath = os.path.join(accel_root_dir, group, save_filename)

        # Save QoM data to file
        print(f"  Saving QoM data to {save_filepath}")
        qom_df.to_csv(save_filepath, index=False)
        print("  Saved QoM data.")
        print("======")